In [1]:
import rsudp.raspberryshake as rs
from obspy.core.stream import Stream
import obspy as obs
from obspy import read

import matplotlib
%matplotlib qt # make sure you have the PyQT package installed in the environment
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
from IPython.display import display, clear_output
from matplotlib.widgets import Button
import pickle
import io

In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/sjaak/miniconda3/envs/rsudp/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/sjaak/m

In [2]:
rs.initRSlib() # running this twice in a row gives an error. Restart kernel in that case
s = Stream()


[get_inventory] WARNING: No station name given, continuing without inventory.


In [4]:
components = ['Z','N']#,'E'] # components to plot. e.g. only Z, or all three Z, N, E

cl = len(components)
fig = plt.figure()
if 'Z' in components:
    ax1 = fig.add_subplot(cl, 1, 1) # Z-component 
if 'E' in components:
    ax2 = fig.add_subplot(cl, 1, cl-1) # E-component
if 'N' in components:
    ax3 = fig.add_subplot(cl, 1, cl) # N-component

# Define functions for pausing, resuming and duplicating animation
pause = False
class Pausing: 
    def pause(self, event):
        global pause
        pause = True

    def resume(self, event):
        global pause
        pause = False

    def duplicate(self, event): # duplicate one timeframe of the traces
        # inx =  # ax number 3 (N component)               
        # if not event.inaxes: return
        # inx = list(fig.axes).index(event.inaxes)
        buf = io.BytesIO()
        pickle.dump(fig, buf)
        buf.seek(0)
        fig2 = pickle.load(buf)         
        for i, ax in enumerate(fig2.axes):
            if i > len(components)-1: # delete the button axes for duplicated figure
                fig2.delaxes(ax)
            else:
                axes=ax
        # axes.change_geometry(1,1,1)
        fig2.show() # still need to reshape figure nicely

# Create buttons to interact with figure
callback = Pausing() 
axpause = fig.add_axes([0.340, 0.9, 0.1, 0.075])
axresume = fig.add_axes([0.450, 0.9, 0.1, 0.075])
axdupl = fig.add_axes([0.560, 0.9, 0.1, 0.075])
bpause = Button(axpause, 'Pause')
bpause.on_clicked(callback.pause)
bresume = Button(axresume, 'Resume')
bresume.on_clicked(callback.resume)
bdupl = Button(axdupl, 'Duplicate')
bdupl.on_clicked(callback.duplicate)

0

In [5]:
# plt.rcParams['fig.labelsize'] = 'x-large'
# plt.rcParams('axes', labelsize='20')    # fontsize of the x and y labels

# This function is called periodically from FuncAnimation
import rsudp.raspberryshake as rs
import matplotlib.pyplot as plt

def animate(i):
    global pause # necessary to declare global?
    global components # idem? 
    """
    Function to be called by animation.FuncAnimation(). This function provides 
    for each frame to be plotted the relevant data. Plots Z, N and E-component data 
    retrieved from the connected Raspberry Shake seismometer. 
    Each time rs.getDATA() is called, only one component is updated. Therefore we call
    it three times to update all 3 components. 
    """
    for i in range(3):
        d = rs.getDATA() # regardless of 'pause', we keep updating the data
        if not pause:    
            rs.update_stream(s, d) # if pause = True, we stop writing data to plotting 
                                   # variable
    trE = s.select(channel="EHE")[0] # index [0] to obtain only trace present inside stream
    trN = s.select(channel="EHN")[0]
    trZ = s.select(channel="EHZ")[0]

    fig.canvas.draw()
    fig.canvas.flush_events()

    # Define time window for plotting
    et = trZ.stats.endtime
    delta=10 # time window in seconds 
    newet = et-delta
    trZ = trZ.slice(starttime=newet, endtime=et) # trim time-window of data
    trE = trE.slice(starttime=newet, endtime=et)
    trN = trN.slice(starttime=newet, endtime=et)

    if 'Z' in components: # plot Z-trace
        ax1.clear() 
        ax1.plot(trZ.times("matplotlib"), trZ.data, "k-", lw=0.5)
        ax1.xaxis_date()
        ax1.legend(['Vertical component [AM.Z0000.00.EHZ]'], loc='upper left')
        # ax1.set_xlabel('Time')

    if 'E' in components: # plot E-trace
        ax2.clear() 
        ax2.plot(trE.times("matplotlib"), trE.data, "k-", lw=0.5)
        ax2.xaxis_date()
        ax2.legend(['East-West component [AM.Z0000.00.EHE]'], loc='upper left')
        ax2.set_xlabel('Time')#, fontsize=)
        ax2.set_ylabel('Amplitude')

    if 'N' in components: # plot N-trace
        ax3.clear() 
        ax3.plot(trN.times("matplotlib"), trN.data, "k-", lw=0.5)
        ax3.xaxis_date()
        ax3.legend(['North-South component [AM.Z0000.00.EHN]'], loc='upper left')
        ax3.set_xlabel('Time')

    if 'Z' in components and 'E' in components:
        ax1.sharey(ax2)
    if 'E' in components and 'N' in components:
        ax3.sharey(ax2)    
    fig.autofmt_xdate()  
    plt.xticks(rotation=45, ha='right')

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(fig, animate, interval=1)
plt.show()